In [79]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import pandas as pd
import csv

# Import matplotlib libraries
from matplotlib import pyplot as plt

In [80]:
import os
import numpy as np
import cv2
from glob import glob

def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")
def save_frame(video_path, gap=30):
    
    save_path="frames"
    create_dir(save_path)
#     print("yes")

    cap = cv2.VideoCapture(video_path)
#     print("yes again")
    #fps = cap.get( cv2.CAP_PROP_FPS ) 
    #print(fps)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break

        if (idx>=15 and (idx-gap//2) % gap == 0):
            #image=img_crop(frame)
            cv2.imwrite(f"{save_path}/{idx}.png", frame)

        idx += 1
if __name__ == "__main__":
    video_paths = glob("D:/BE/Project/test_videos/*")
    print(video_paths)
#     save_dir = "geet_dataset2/train/"
    for path in video_paths:
        save_frame(path, gap=60)
    

['D:/BE/Project/test_videos\\nidhip.mp4']


In [81]:
def image_to_movenet(path):
    image_path =path
    image = tf.io.read_file(image_path)
    image = tf.compat.v1.image.decode_jpeg(image)
    image = tf.expand_dims(image, axis=0)
    # Resize and pad the image to keep the aspect ratio and fit the expected size.
    image = tf.cast(tf.image.resize_with_pad(image, 192, 192), dtype=tf.int32)

    # Download the model from TF Hub.
    model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    movenet = model.signatures['serving_default']

    # Run model inference.
    outputs = movenet(image)
# Output is a [1, 1, 17, 3] tensor.
    keypoints = outputs['output_0']
    kpts_x = keypoints[0, 0, :, 0]
    kpts_y = keypoints[0, 0, :, 1]
    kpts_scores = keypoints[0, 0, :, 2]
    
    
    

     # Empty dataset
    t_x=kpts_x.numpy()
    t_y=kpts_y.numpy()
    
    tn=[]
   
    for i in range(0,17):
        tn.append(t_x[i])
        tn.append(t_y[i])
    return tn

In [82]:
import os
import time
from collections import defaultdict

path_lst=[]

keypts_inner_test=[]
keypts_outer_test=[]



for image in os.listdir('frames'):
    path_lst.append(os.path.abspath(f"frames/{image}"))
        
        #path_lst_per.append(path_lst)
        #image_path_dictionary[name] = path_list
              
path_lst = sorted(path_lst,key=os.path.getmtime)

for path in path_lst:
    print(path)
    keypts_inner_test=image_to_movenet(path)
    keypts_outer_test.append(keypts_inner_test)

D:\BE\Project\Yoga_Class-Analysis\frames\30.png
D:\BE\Project\Yoga_Class-Analysis\frames\90.png
D:\BE\Project\Yoga_Class-Analysis\frames\150.png
D:\BE\Project\Yoga_Class-Analysis\frames\210.png
D:\BE\Project\Yoga_Class-Analysis\frames\270.png
D:\BE\Project\Yoga_Class-Analysis\frames\330.png
D:\BE\Project\Yoga_Class-Analysis\frames\390.png


In [83]:
print(keypts_outer_test)

[[0.40834564, 0.5285305, 0.39134464, 0.5442334, 0.38715324, 0.5182702, 0.37302798, 0.55452317, 0.3697573, 0.4980904, 0.39781296, 0.5772733, 0.4077251, 0.45367104, 0.4871038, 0.6016332, 0.50545853, 0.46478093, 0.5489899, 0.62707263, 0.58423316, 0.5034302, 0.55806273, 0.55844676, 0.5552764, 0.47660667, 0.6060969, 0.6352718, 0.5743242, 0.38505667, 0.6207972, 0.5047588, 0.6218187, 0.5169618], [0.4081831, 0.5374985, 0.3906402, 0.54796886, 0.3895032, 0.5239401, 0.3733064, 0.5587251, 0.37315452, 0.50427896, 0.40029693, 0.5769456, 0.40843612, 0.46444452, 0.49382967, 0.6182232, 0.50727326, 0.44754368, 0.5183125, 0.64773184, 0.5217031, 0.5506582, 0.5507156, 0.5506505, 0.5520171, 0.47612762, 0.57405996, 0.60297275, 0.55776083, 0.45229012, 0.6134355, 0.50586057, 0.60618657, 0.5130745], [0.39216763, 0.4844072, 0.37958443, 0.49777555, 0.38130778, 0.47490558, 0.36241361, 0.52178633, 0.3702756, 0.4706586, 0.40500474, 0.55737007, 0.40208387, 0.445261, 0.52325904, 0.5519822, 0.503337, 0.42675436, 0.6047

In [84]:
from keras.layers import Masking
def padding_masking(x_test):
    testing=[]
    for i in range(0,1):
        result = np.zeros((77, 34))
        result[:np.array(x_test[0]).shape[0],:np.array(x_test[0]).shape[0]]=x_test[0]
        testing.append(result)
    x_test=tf.convert_to_tensor(testing,dtype='float32')
    mask_layer=Masking(mask_value=0.0)
    x_test=mask_layer(x_test)
    return x_test

In [85]:
x_test=keypts_outer_test
x_test=np.array(x_test)
x_test.shape[0]
x_test=padding_masking(x_test)

In [86]:
print(x_test.shape)

(1, 77, 34)


load model


In [87]:
from tensorflow import keras
from keras.layers import Bidirectional
from keras.layers import LSTM
# It can be used to reconstruct the model identically.
model = keras.models.load_model("savemodel.h5")

# Let's check:
# np.testing.assert_allclose(
#     model.predict(test_input), reconstructed_model.predict(test_input)
# )

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:

y_pred = model.predict(x_test)
print(y_pred)

[[-2.6049933  6.7246857 -1.5302136 -2.9431598  3.8076189 -3.437937 ]]
